In [1]:
! pip install accelerate

In [2]:
! pip install transformers
! pip install torch
! pip install simplet5 -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda 23.7.3 requires setuptools>=60.0.0, but you have setuptools 59.5.0 which is incompatible.
opentelemetry-api 1.18.0 requires importlib-metadata~=6.0.0, but you have importlib-metadata 6.7.0 which is incompatible.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.23.5 which is incompatible.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scipy 1.11.2 which is incompatible.


In [3]:
import numpy as np
import pandas as pd
import torch

In [4]:
df = pd.read_csv("/kaggle/input/imbd-reviews-classification/train.tsv")
df.head()

,text,sentiment
0,Another cult strikes again. This isn't a spoil...,0
1,This must be one of the worst movies I've ever...,0
2,"I was one of those ""few Americans"" that grew u...",0
3,I saw this movie in my international cinema cl...,0
4,"I just sat in the theater bored as hell, i wan...",0


In [5]:
df['sentiment'] = df['sentiment'].astype("str").apply(lambda x: x.replace("0","negative") if "0" == x else x.replace("1", "positive"))
df.head()

,text,sentiment
0,Another cult strikes again. This isn't a spoil...,negative
1,This must be one of the worst movies I've ever...,negative
2,"I was one of those ""few Americans"" that grew u...",negative
3,I saw this movie in my international cinema cl...,negative
4,"I just sat in the theater bored as hell, i wan...",negative


In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base").to("cuda")

# input_text = "Please classify the following text as either negative or positive:" + df['text'][0]
# input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

# outputs = model.generate(input_ids)
# print(tokenizer.decode(outputs[0]))
# print(input_text)

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [7]:
max_input_length = 512
max_label_length = 16

In [8]:
df['text'] = "Please classify the following text as either negative or positive:" + df['text']
df['sentiment'] = "labels :" + df['sentiment']
df.head()

,text,sentiment
0,Please classify the following text as either n...,labels :negative
1,Please classify the following text as either n...,labels :negative
2,Please classify the following text as either n...,labels :negative
3,Please classify the following text as either n...,labels :negative
4,Please classify the following text as either n...,labels :negative


In [9]:
from torch.utils.data import DataLoader, Dataset
# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length4text=512, max_length4label=32):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length4text = max_length4text
        self.max_length4label = max_length4label

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data['text'][idx]
        label = self.data['sentiment'][idx]
        text_encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length4text, return_tensors='pt').to("cuda")
        label_encoding = self.tokenizer(label, truncation=True, padding='max_length', max_length=self.max_length4label, return_tensors='pt').to("cuda")
        return {
            'input_ids': text_encoding['input_ids'].squeeze(),
            'attention_mask': text_encoding['attention_mask'].squeeze(),
            'input_id_labels': label_encoding['input_ids'].squeeze(),
            'attention_mask_labels': label_encoding['attention_mask'].squeeze()
        }

In [10]:
# Create a dataset and data loader
batch_size = 1
train_dataset = CustomDataset(df[:50], tokenizer, max_length4text=max_input_length, max_length4label=max_label_length)
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Define the loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [11]:
model.train()
for epoch in range(1):
    for batch in train_data_loader:
        inputs = batch['input_ids']
        attention_mask = batch['attention_mask']
        input_id_labels = batch['input_id_labels']
        attention_mask_labels = batch['attention_mask_labels']

        optimizer.zero_grad()
        outputs = model(inputs, attention_mask=attention_mask, labels=input_id_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

In [47]:
print(loss)

tensor(0.0002, device='cuda:0', grad_fn=<NllLossBackward0>)


In [16]:
test_df = pd.read_csv("/kaggle/input/imbd-reviews-classification/test.tsv")
test_df['sentiment'] = test_df['sentiment'].astype("str").apply(lambda x: x.replace("0","negative") if "0" == x else x.replace("1", "positive"))
test_df['text'] = "Please classify the following text as either negative or positive:" + test_df['text']
test_df['sentiment'] = "labels :" + test_df['sentiment']
test_df.head()

,text,sentiment
0,Please classify the following text as either n...,labels :negative
1,Please classify the following text as either n...,labels :negative
2,Please classify the following text as either n...,labels :negative
3,Please classify the following text as either n...,labels :negative
4,Please classify the following text as either n...,labels :negative


In [43]:
from sklearn.metrics import accuracy_score
# Create a dataset and data loader for the test data
batch_size = 1
test_dataset = CustomDataset(test_df[:500], tokenizer, max_length4text=max_input_length, max_length4label=max_label_length)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)  

# Put the model in evaluation mode
model.eval()

# Lists to store predictions and true labels
all_predictions = []
all_true_labels = []

# Iterate over the test data
for batch in test_data_loader:
    inputs = batch['input_ids']
    attention_mask = batch['attention_mask']
    input_id_labels = batch['input_id_labels']
    attention_mask_labels = batch['attention_mask_labels']
    

    with torch.no_grad():
        outputs = model.generate(inputs, attention_mask=attention_mask, max_length=32)
        predictions = [tokenizer.decode(output) for output in outputs]

    all_predictions.extend(predictions)
    all_true_labels.extend([tokenizer.decode(input_id_label) for input_id_label in input_id_labels])

# Calculate accuracy



In [44]:
correct_labels = [x.split(":")[1].split("<")[0] for x in all_true_labels]
print(correct_labels[0])

negative


In [45]:
prediction_labels = [x.split(":")[1].split("<")[0] for x in all_predictions]
print(prediction_labels[0])

negative


In [46]:

accuracy = accuracy_score(correct_labels, prediction_labels)
print(f"Accuracy: {accuracy}")

Accuracy: 1.0
